In [7]:
from dotenv import load_dotenv

from langchain.tools import Tool
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_community.utilities import TextRequestsWrapper

from bs4 import BeautifulSoup

#from src.search import top_n_results_factory

In [8]:
load_dotenv()

True

In [9]:
def top_3_results(query):
    search = GoogleSearchAPIWrapper()
    return search.results(query, 3)

In [10]:
tool = Tool(
    name="Google Search",
    description="Search Google for recent results.",
    func=top_3_results,
)

In [11]:
searches = tool.run("Kubanda")

In [12]:
searches

[{'title': 'First tests of Timepix detectors based on semi-insulating GaAs ...',
  'link': 'https://ui.adsabs.harvard.edu/abs/2018JInst..13C2013Z/abstract',
  'snippet': 'Kubanda, D. ;; Žemlička, J. ;; Šagátová, A. ;; Zápražný, Z. ;; Boháček, P. ;; Nečas, V. ;; Mora, Y. ;; Pichotka, M. ;; Dudák, J. Abstract. In this work, we have\xa0...'},
 {'title': 'Spectrometric properties of semi-insulating GaAs detectors ...',
  'link': 'https://ui.adsabs.harvard.edu/abs/2016JInst..11C2078S/abstract',
  'snippet': 'Fülöp, M. ;; Kubanda, D. ;; Nečas, V. Abstract. The radiation hardness of Semi-Insulating (SI) GaAs detectors against 5 MeV electrons is investigated in this\xa0...'},
 {'title': 'Semi-insulating GaAs based detector of fast neutrons produced by ...',
  'link': 'https://ui.adsabs.harvard.edu/abs/2016JInst..11C2002S/abstract',
  'snippet': "Šagátová, A. ;; Kubanda, D. ;; Zat'ko, B. ;; Sedlačková, K. ;; Nečas, V. ;; Solar, M. ;; Granja, C. Abstract. We have examined semi-insulating (SI) Ga

In [13]:
links = [search['link'] for search in searches]

In [14]:
links

['https://ui.adsabs.harvard.edu/abs/2018JInst..13C2013Z/abstract',
 'https://ui.adsabs.harvard.edu/abs/2016JInst..11C2078S/abstract',
 'https://ui.adsabs.harvard.edu/abs/2016JInst..11C2002S/abstract']

In [15]:
requests = TextRequestsWrapper()

In [16]:
html_list = [requests.get(link) for link in links]

In [17]:
type(html_list[0])

str

In [18]:
soup = BeautifulSoup(html_list[0])

In [19]:
print(soup.find('meta', property='og:title')["content"])

First tests of Timepix detectors based on semi-insulating GaAs matrix of different pixel size


In [20]:
# Get first 3 authors (remove comma from name) and append et.al. if more than 3 and format as a string
authors = soup.find_all("meta", property="article:author")
authors = [author["content"].replace(",", "") for author in authors]
if len(authors) > 3:
    authors = authors[:3]
    authors.append("et al.")
authors = ", ".join(authors)
authors

'Zaťko B., Kubanda D., Žemlička J., et al.'

In [21]:
soup.find_all("meta", property="article:author")

[<meta content="Zaťko, B." property="article:author"/>,
 <meta content="Kubanda, D." property="article:author"/>,
 <meta content="Žemlička, J." property="article:author"/>,
 <meta content="Šagátová, A." property="article:author"/>,
 <meta content="Zápražný, Z." property="article:author"/>,
 <meta content="Boháček, P." property="article:author"/>,
 <meta content="Nečas, V." property="article:author"/>,
 <meta content="Mora, Y." property="article:author"/>,
 <meta content="Pichotka, M." property="article:author"/>,
 <meta content="Dudák, J." property="article:author"/>]

In [22]:
abstract = soup.find('meta', property='og:description')["content"]
print(abstract)

In this work, we have focused on Timepix detectors coupled with the semi-insulating GaAs material sensor. We used undoped bulk GaAs material with the thickness of 350 μm. We prepared and tested four pixelated detectors with 165 μm and 220 μm pixel size with two versions of technology preparation, without and with wet chemically etched trenches around each pixel. We have carried out adjustment of GaAs Timepix detectors to optimize their performance. The energy calibration of one GaAs Timepix detector in Time-over-threshold mode was performed with the use of <SUP>241</SUP>Am and <SUP>133</SUP>Ba radioisotopes. We were able to detect γ-photons with the energy up to 160 keV. The X-ray imaging quality of GaAs Timepix detector was tested with X-ray source using various samples. After flat field we obtained very promising imaging performance of tested GaAs Timepix detectors.


In [23]:
from langchain.prompts import PromptTemplate

In [28]:
prompt_template = PromptTemplate.from_template(
    """
    You are a seasoned researcher, teacher and a popularizer of science that can explain scientific concepts to the wide variety of audiences.
    You will be given an abstract of a scientific paper, an audience (e.g. high-school students) and a language, and you will have to explain the scientific paper it in a way that is understandable for the audience and translate that explanation to given language. 
    Please be concise and clear, make a summary and few bulletpoints. Please only include the translation without any introductory sentences.
    
    Audience: {audience}.
    
    Language: {language}.
    
    Abstract: {abstract}
    """
)

In [29]:
my_prompt = prompt_template.format(audience="middle school students", abstract=abstract, language="Slovak")

In [30]:
print(my_prompt)


    You are a seasoned researcher, teacher and a popularizer of science that can explain scientific concepts to the wide variety of audiences.
    You will be given an abstract of a scientific paper, an audience (e.g. high-school students) and a language, and you will have to explain the scientific paper it in a way that is understandable for the audience and translate that explanation to given language. 
    Please be concise and clear, make a summary and few bulletpoints. Please only include the translation without any introductory sentences.
    
    Audience: middle school students.
    
    Language: Slovak.
    
    Abstract: In this work, we have focused on Timepix detectors coupled with the semi-insulating GaAs material sensor. We used undoped bulk GaAs material with the thickness of 350 μm. We prepared and tested four pixelated detectors with 165 μm and 220 μm pixel size with two versions of technology preparation, without and with wet chemically etched trenches around each p

In [35]:
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI


In [36]:
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", streaming=True)


In [38]:
for chunk in llm.stream(my_prompt):
    print(chunk.content, end="", flush=True)

V tejto práci sme sa zameriavali na detektory Timepix spojené so senzorom z polovodičového materiálu GaAs. Použili sme nedotovaný blokový materiál GaAs s hrúbkou 350 μm. Pripravili sme a otestovali štyri pixelové detektory s veľkosťou pixelu 165 μm a 220 μm s dvoma verziami technologickej prípravy, bez a s mokrým chemickým leptaním priekop okolo každého pixelu. Vykonali sme nastavenie detektorov GaAs Timepix na optimalizáciu ich výkonu. Energetická kalibrácia jedného detektora GaAs Timepix v móde Time-over-threshold bola vykonaná pomocou rádioizotopov <SUP>241</SUP>Am a <SUP>133</SUP>Ba. Boli sme schopní detekovať γ-fotóny s energiou až do 160 keV. Kvalita röntgenového zobrazovania detektora GaAs Timepix bola testovaná pomocou röntgenového zdroja s použitím rôznych vzoriek. Po vyrovnávacej ploche sme dosiahli veľmi sľubný výkon zobrazovania testovaných detektorov GaAs Timepix.

- Skúmali sme detektory Timepix spojené so senzorom z materiálu GaAs.
- Použili sme nedotovaný blokový materi